# Transformer 

- [Attention Is All You Need 2017](https://arxiv.org/pdf/1706.03762) 논문에서 제시한 모델로 RNN모듈을 사용하지 않고 **Attention 모듈로만 구성한 Encoder-Decoder 구조의 seq2seq 모델**이다.
    - RNN → RNN + attention → Transformer
- Transformer 이후 자연어 관련 pretrained 모델들은 대부분 Transformer의 Encoder나 Decoder를 변형하거나 단순히 층을 깊게 쌓는 형태로 구현하여 성능을 올렸다.
    - Google의 BERT(Bidirectional Encoder Representations from Transformers)는 Transformer의 Encoder를 이용해 구현했다.
    - OpenAI의 GPT(Generative Pre-trained Transformer)는 Transformer의 Decoder를 이용해 구현했다.

## Seq2Seq의 문제와 Transformer
- Seq2Seq 의 문제
    - 장기기억 소실의 문제.(long term dependency problem)
        - seq가 길어지면 초기 timestep의 정보가 점점 소실된다.
    - 병렬처리를 못해 연산 효율이 떨어진다.
        - time step 별로 순서대로 처리해야 하기 때문에 병렬처리가 안된다.
- **장기기억 소실문제는** seq2seq에 **Attention mechanism**을 적용해 해결할 수 있다.
  
![img](figures/transformer_advantage.gif)
- Transformer는  encoder와 decoder에서 RNN을 제거하고 **Attention 모듈로 변경하여 병렬 처리가 가능**하도록 했다.

## Transformer Architecture
- Encoder - Decoder  구조
- Encoder와 Decoder가 **self-attention 모듈**로 구성된다. 그래서 RNN을 쓰지 않고 입력 sequence와 출력 sequence를 처리한다.
- **Positional Encoding**
    - RNN을 사용하지 않기 때문에 입력 sequence의 토큰들에 순서정보가 없다. 그 순서정보를 Embedding에 추가하는 역할.
    - 문장을 구성하는 각 토큰들의 embedding vector에 그 토큰 순서 정보를 가지는 positional encoding 값을 더해준다.
    - Positional Encoding 값은 sine/cosin 주기함수를 이용해 계산한다.

![img](figures/transformer_architecture2.png)

\[transformer의 구조 1\]

### ![img](figures/transformer_architecture1.png)

\[Encoder와 Decoder의 구조\]

## Self Attention
- Attention을 자기 자신한테 적용한다는 의미
    - seq2seq 의 attention은 decoder의 sequence가 encoder의 hidden state에 attention을 취한다.
    - self attention은 attention을 자기 자신에게 취한다. 
- Self-attention은 한 문장에서 하나의 단어가 다른 단어들과 어떤 연관성을 가지는 지를 찾는다.

![img](figures/transformer_self-attention.png)

예를 들어 "The animal didn't cross the street because it was too tired." 라는 문서에서 `it`을 encoding 한려고 한다. `it`을 이 문서의 문맥에 맞게 embedding하기 위해서는 이 문장내에서 어떤 의미로 쓰였는지 알아야 한다. 그것을 알기 위해서는 `it` 이 가리키는 것이 어떤 것인지 알아야 한다. 그것을 다른 문서에서 찾는 것이 아니라 `it` 있는(대상 단어가 있는) 문서에서 다른 단어들과의 연관성에서 찾는다. 그래서 **의미를 파악하려는 문서와 그 의미를 찾는 문서가 동일하기 때문에 self-attention**이라고 한다.

### Self Attetion 과정
- 같은 문장내에서 단어들 간의 관계를 고려하는 것.
  
![self-attention](figures/transformer_self_attention.png)

- **수식**
\begin{align}
&\text{Attention(Q,K,V)} = \text{softmax}(\cfrac{QK^T}{\sqrt{d_k})})V \\
&\small \text{Q: Query, K: Key, V: Value,}\;d_{k}: \text{emmbeding 차원}
\end{align}

1. Query, Key, Value 생성
    - **Query**: Embedding vector를 구할 대상 (ex: it)
    - **Key**: Query와 연관성있는 단어들을 계산할 대상 (ex: 위 그림의 왼쪽 문서 토큰들)
    - **Value**: Query와 Key를 이용해 찾은 attention weight를 적용해 Attention value를 찾을 대상. 
    - Self attention은 Query, Key, Value 모두 입력 Sequence(X)로 부터 생성한다.
        - X에 각각다른 Weight를 내적하여 만든다.
       - $Q=X\cdot W_q$ , $K=X\cdot W_k$ , $V=X\cdot W_v$
![img](figures/transformer_query_key_value.png)

2. **Attention Score**
    - Query 와 Key를 내적(dot product)하여 유사도를 계산한다. embedding vector의 차원의 제곱근으로 나눠서 정규화한다.
    - Scaled Dot Product Attention
   $$
   \text{Attention Score} = \cfrac{Q\cdot K^T}{\sqrt{d_k}}
   $$
![img](figures/transformer_query_key_matmulpng.png)

4. **Attention Weight(Distribution)**
   - 위에서 계산된 Attention score에 softmax를 적용해 0 ~ 1 사이 비율로 바꾼다.
     
   $$
    \text{Attention Weight} = softmax(\text{Attention Score})
   $$

5. **Attention value**
   - 최종 attention 연산의 결과로 **Attention weight를 Value에 내적**해서 Attention Value를 만든다.
   - Attention은 Attention value값을 **입력 sequence의 context vector**가 된다.
   $$
   \text{Attention Value} = \text{Attention Weight}\cdot\text{Value}
   $$ 
![img](figures/transformer_attention_value.png)

# Multi-Head attention
- Multi-Head Attention 모듈에서는 여러 개의 Self-Attention Layer을 사용해 입력 sequence 에 대한 여러 feature들을 추출한다.
- 여러 Self attention layer들이 출력한 결과들을 합쳐서(concatenate) Linear Layer를 통과시켜 shape을 맞춘 뒤 최종 output context vector를 만든다.

![img](figures/transformer_multi-head-attention.png)

# Masked self Attention
- 입력 sequence 가 순서대로 입력되어 처리 되는 경우 i번째 입력 단어의 경우 그 이후의 단어는 모르는 상태이다. 그러나 attention은 입력된 모든 토큰을 한번에 처리해버리기 때문에 attention score를 구할 수있게 된다. 이것은 주어진 토큰(i번째)이 미래시점의 입력토큰(i+1 이후 번째)과의 유사도를 계산한 것이 된다. 이 문제를 해결하기 위해 i번째 토큰에 대한 attention score는 i번째 까지의 토큰들과 계산하도록 한다.
- Attention Score를 Softmax 계산하기 전에 적용한다.
- Attention  계산시 선택적으로 mask를 사용하면 Masked self attention이 된다. (Masked Multi-Head Attention)

![img](figures/transformer_masked_self_attention.png)

In [2]:
import torch
a = torch.tensor([1.0, -torch.inf, -torch.inf, -torch.inf])
a.softmax(dim=-1)

tensor([1., 0., 0., 0.])